In [1]:
import pandas as pd
import numpy as np

In [3]:
# Csv reading
df_training = pd.read_csv('D:\MSc Aegean\Thesis\Weka\Train_Test_Sets\Sorted Set\Train_70.csv', usecols = ['subject_id', 'hadm_id', 'date', 'hospital_expire_flag'])

df_test = pd.read_csv('D:\MSc Aegean\Thesis\Weka\Train_Test_Sets\Sorted Set\Test_30.csv', usecols = ['subject_id', 'hadm_id', 'date', 'hospital_expire_flag'])

df_patients = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\core\patients.csv.gz')

df_admissions = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\core\Admissions.csv.gz', usecols = ['subject_id', 'ethnicity'])

In [4]:
# Μετατροπή του date από Object σε datetime
df_training['date']= pd.to_datetime(df_training['date'])
df_test['date']= pd.to_datetime(df_test['date'])

In [5]:
#Απομόνωση της χρονιάς από το date

df_training['training_set_year'] = df_training['date'].dt.year

df_test['training_set_year'] = df_test['date'].dt.year

In [6]:
# Κρατάμε τις μοναδικές εγγραφές
tempI = df_training.drop_duplicates(subset=['subject_id'])

tempII = df_test.drop_duplicates(subset=['subject_id'])

df_admissions = df_admissions.drop_duplicates(subset=['subject_id'])

In [7]:
#Ενοποίηση πινάκων patients με τα training & test set
rslt_df_training=pd.merge(tempI, df_patients, on=['subject_id'], how='left')

rslt_df_test=pd.merge(tempII, df_patients, on=['subject_id'], how='left')

# Προσθήκη και του admissions

rslt_df_training=pd.merge(rslt_df_training, df_admissions, on=['subject_id'], how='left')

rslt_df_test=pd.merge(rslt_df_test, df_admissions, on=['subject_id'], how='left')

In [8]:
"""
Υπολογισμός διαφοράς του έτους που έχει δηλωθεί
στον ασθενή σε σχέση με το έτος εισαγωγής.
"""

rslt_df_training['temp_age'] = rslt_df_training['training_set_year'] - rslt_df_training['anchor_year']

rslt_df_test['temp_age'] = rslt_df_test['training_set_year'] - rslt_df_test['anchor_year']

In [9]:
# Πρόσθεση τις διαφοράς έτους με την ηλικία που είχε αρχικά δηλωθεί.
rslt_df_training['Final_Age'] = rslt_df_training['temp_age'] + rslt_df_training['anchor_age']
rslt_df_test['Final_Age'] = rslt_df_test['temp_age'] + rslt_df_test['anchor_age']

In [9]:
"""
Αντικατάσταση των τιμών ηλικίας με την τιμή 89
σε όλους τους ασθενείς που η ηλικία τους είχε
δηλωθεί 91
"""

#rslt_df_training.loc[rslt_df_training.anchor_age == 91, "Final_Age"] = 89
#rslt_df_test.loc[rslt_df_test.anchor_age == 91, "Final_Age"] = 89

'\nΑντικατάσταση των τιμών ηλικίας με την τιμή 89\nσε όλους τους ασθενείς που η ηλικία τους είχε\nδηλωθεί 91\n'

In [13]:
rslt_df_test.to_csv('D:\Paper\Chart_Lab_by_hours\Training_set.csv', index=False)

In [10]:
#Μέσος όρος ηλικίας training set
rslt_df_training['Final_Age'].mean()

67.99260412611912

In [11]:
#Μέσος όρος ηλικίας test set
rslt_df_test['Final_Age'].mean()

68.52460538532962

In [12]:
# Μέσος όρος ηλικίας από το σύνολο training & test set
(rslt_df_training['Final_Age'].sum() + rslt_df_test['Final_Age'].sum()) / (rslt_df_training.shape[0] + rslt_df_test.shape[0])

68.14975315414152

In [13]:
# Φύλλο στο training set
rslt_df_training['gender'].value_counts()

M    1362
F    1207
Name: gender, dtype: int64

In [14]:
# Φύλλο στο test set
rslt_df_test['gender'].value_counts()

M    560
F    517
Name: gender, dtype: int64

In [15]:
# Φύλλο στο σύνολο training set & test set
rslt_df_training['gender'].value_counts() + rslt_df_test['gender'].value_counts()

M    1922
F    1724
Name: gender, dtype: int64

In [16]:
# Εθνικότητα στο training set
rslt_df_training['ethnicity'].value_counts()

WHITE                            1785
UNKNOWN                           222
BLACK/AFRICAN AMERICAN            201
OTHER                             135
HISPANIC/LATINO                    95
ASIAN                              70
UNABLE TO OBTAIN                   52
AMERICAN INDIAN/ALASKA NATIVE       9
Name: ethnicity, dtype: int64

In [17]:
#Εθνινκότητα στο test set
rslt_df_test['ethnicity'].value_counts()

WHITE                            751
UNKNOWN                          113
BLACK/AFRICAN AMERICAN            86
OTHER                             53
HISPANIC/LATINO                   30
ASIAN                             24
UNABLE TO OBTAIN                  17
AMERICAN INDIAN/ALASKA NATIVE      3
Name: ethnicity, dtype: int64

In [18]:
# Εθνικότητα στο σύνολο training & test set
rslt_df_training['ethnicity'].value_counts() + rslt_df_test['ethnicity'].value_counts()

WHITE                            2536
UNKNOWN                           335
BLACK/AFRICAN AMERICAN            287
OTHER                             188
HISPANIC/LATINO                   125
ASIAN                              94
UNABLE TO OBTAIN                   69
AMERICAN INDIAN/ALASKA NATIVE      12
Name: ethnicity, dtype: int64

In [19]:

rslt_df_test['anchor_age'].where(rslt_df_training['anchor_age'] == 91).count()

40